# Ensemble Learning

## Initial Imports

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [23]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [24]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [25]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [26]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [27]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [28]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,y
    ,random_state=1
    , stratify=y
)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [29]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler_std = StandardScaler()

In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

def Categorize_Txt_Cols(df,col_name):
    """Categorizes the columns with numbers"""
    le.fit(df[col_name])
    return le.transform(df[col_name])

def Get_Columns_With_Text(df):
    """Get's alist of text columns from a DataFrame"""
    list_of_columns = list(zip(list(df.keys()), df.dtypes.to_list()))
    return [col[0] for col in list_of_columns if col[1]==np.dtype('O')]

def Convert_Text_Cols(df):
    """Determines which columns are Text"""
    list_of_text_columns = Get_Columns_With_Text(df)
    
    for col in list_of_text_columns:
        df[col] = Categorize_Txt_Cols(df,col_name=col)
    
    return df


In [31]:
# Categorize Text Columns
X_train = Convert_Text_Cols(X_train)
X_test  = Convert_Text_Cols(X_test)


In [32]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler_std = scaler_std.fit(X_train)


In [33]:
# Scale the training and testing data
X_train_scaled_std = X_scaler_std.transform(X_train)
X_test_scaled_std  = X_scaler_std.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [34]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [40]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_brf = brf.predict(X_test)
print(f"Balanced accuracy Score: {balanced_accuracy_score(y_test, y_pred_brf):%}")

Balanced accuracy Score: 77.452047%


In [64]:
confusion_matrix(y_test, y_pred_brf)
sorted(set(y_pred_brf))

['high_risk', 'low_risk']

In [87]:
def Combine_Names_for_CF_Matrix(names=None, y_pred=None):
    if names is not None:
        lst = names
    elif y_pred is not None:
        lst = sorted(set(y_pred))

    idx = [f"Actual {x}" for x in lst]
    col = [f"Prediction {x}" for x in lst]

    return idx,col

def Generate_Confusion_Matrix(y_test, y_pred, names=None):
    from sklearn.metrics import confusion_matrix

    idx,col = Combine_Names_for_CF_Matrix(names=names, y_pred=y_pred)

    cm_df = pd.DataFrame(
        data=confusion_matrix(y_test, y_pred)
        , index= idx
        , columns= col
    )
    cm_df['Total Predictions'] = cm_df.sum(axis='columns')
    cm_df.loc["Total Actuals"] = cm_df.sum(axis='index')

    return cm_df
def Calc_Conf_Matrix(cm_df:pd.DataFrame):
    true_negative  = cm_df.iloc[0,0]
    false_positive = cm_df.iloc[0,1]
    false_negative = cm_df.iloc[1,0]
    true_positive  = cm_df.iloc[1,1]
    total_predictions  = cm_df.loc['Total Actuals','Total Predictions']
    total_predict_yes  = cm_df.iloc[2,1]
    total_predict_no     = cm_df.iloc[2,0]
    
    total_actuals_positive = cm_df.iloc[1,2]
    total_actuals_negaitive = cm_df.iloc[0,2]

    accuracy                =  (true_positive  + true_negative ) /total_predictions
    misclassification_rate  =  (false_positive + false_negative) /total_predictions  # Error Rate
    sensitivity_rate        =  true_positive / total_actuals_positive                # True Positive Rate or Recall
    false_positive_rate     =  false_positive / total_actuals_negaitive
    true_negative_rate      =  true_negative  / total_actuals_negaitive              # Specificity
    precision               =  true_positive /  total_actuals_positive

    return {'accuracy':accuracy
            ,'misclassification_rate':misclassification_rate
            ,'sensitivity_rate':sensitivity_rate
            ,'false_positive_rate':false_positive_rate
            ,'true_negative_rate':true_negative_rate
            ,'precision':precision}

def Describe_Confusion_Matrix(cm_df:pd.DataFrame):
    results = Calc_Conf_Matrix(cm_df=cm_df)
    display(cm_df)

    print(f"Overall, how often is the classifier correct (Accuracy): {results['accuracy']:.3%}")
    print(f"Overall, how often is it wrong (Misclassification Rate or Error Rate): {results['misclassification_rate']:.3%}")
    print(f"When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): {results['sensitivity_rate']:.3%}")
    print(f"When it's actually no, how often does it predict no (False Posive Rate): {results['true_negative_rate']:.3%}")
    print(f"When it predicts yes, how often is it correct? (Precision):{results['precision']:.3%}")
    print(f"When it's actually no, how often does it predict no? (False Positive Rate): {results['false_positive_rate']:.3%}")
    

In [88]:
x_df  = Generate_Confusion_Matrix(y_test=y_test, y_pred=y_pred_brf)#.sum(axis='columns')
x_df.iloc[1,1]
Describe_Confusion_Matrix(x_df)

,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,56,31,87
Actual low_risk,1620,15498,17118
Total Actuals,1676,15529,17205


Overall, how often is the classifier correct (Accuracy): 90.403952%
Overall, how often is it wrong (Misclassification Rate or Error Rate): 9.596048%
When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): 90.536278%
When it's actually no, how often does it predict no (False Posive Rate): 64.367816%
When it predicts yes, how often is it correct? (Precision):90.536278%
When it's actually no, how often does it predict no? (False Positive Rate): 35.632184%


In [86]:
x_df

,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,56,31,87
Actual low_risk,1620,15498,17118
Total Actuals,1676,15529,17205


In [36]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm_df = pd.DataFrame(
    data=confusion_matrix(y_test, y_pred_brf)
    , index=["Actual High Risk Loans", "Actual Low Risk Loans"]
    , columns=["Predicted High Risk Loans", "Predicted Low Risk Loans"]
)
display(cm_df)

,Predicted High Risk Loans,Predicted Low Risk Loans
Actual High Risk Loans,56,31
Actual Low Risk Loans,1620,15498


In [37]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test,y_pred_brf, digits=5))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk    0.03341   0.64368   0.90536   0.06353   0.76339   0.56751        87
   low_risk    0.99800   0.90536   0.64368   0.94943   0.76339   0.59801     17118

avg / total    0.99313   0.90404   0.64500   0.94495   0.76339   0.59786     17205



In [38]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
ordered_feature_list = sorted(zip(X_train.columns,importances), key=lambda tup: tup[1], reverse=True)
_ = [print(f"Feature:\t{f[0]:30} with a value of {f[1]:10.4f}") for f in ordered_feature_list if f[1]>0.0]

Feature:	total_rec_prncp                with a value of     0.0719
Feature:	total_rec_int                  with a value of     0.0640
Feature:	total_pymnt_inv                with a value of     0.0604
Feature:	last_pymnt_amnt                with a value of     0.0603
Feature:	total_pymnt                    with a value of     0.0557
Feature:	int_rate                       with a value of     0.0294
Feature:	issue_d                        with a value of     0.0228
Feature:	dti                            with a value of     0.0185
Feature:	installment                    with a value of     0.0175
Feature:	annual_inc                     with a value of     0.0172
Feature:	max_bal_bc                     with a value of     0.0170
Feature:	mths_since_recent_inq          with a value of     0.0164
Feature:	out_prncp_inv                  with a value of     0.0161
Feature:	il_util                        with a value of     0.0160
Feature:	out_prncp                      with a value of     0.

### Easy Ensemble Classifier

In [ ]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [168]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import EasyEnsembleClassifier 

X, y = make_classification(
    n_classes=2, class_sep=2,
    weights=[0.1, 0.9], 
    n_informative=3, 
    n_redundant=1, 
    flip_y=0,
    n_features=20, 
    n_clusters_per_class=1, 
    n_samples=1000, 
    random_state=10
)
X,y

print('Original dataset shape %s' % Counter(y))

X_train, X_test, y_train, y_test = 
        train_test_split(X, y,
                        random_state=0)

eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_train, y_train) 

y_pred = eec.predict(X_test)
print(confusion_matrix(y_test, y_pred))

SyntaxError: invalid syntax (<ipython-input-168-89dfdc890a32>, line 22)